In [1]:
from keys import OPENAI_API_KEY
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.6)
user = {
    'name': 'Charles',
    'age': 21,
    'sex': 'male'
}

In [3]:
from gcal import get_day_events
from weather import fetch_weather_forecast

events = get_day_events()
forecast_data = fetch_weather_forecast(
    latitude=37.8716,
    longitude=-122.2728,
    hourly="temperature_2m,precipitation_probability,precipitation",
    daily="temperature_2m_max,temperature_2m_min,sunrise,sunset",
    temperature_unit="fahrenheit",
    timezone="America/Los_Angeles",
    forecast_days=1
)

Getting the events for 2023-08-31T00:00:00-07:00
[{'kind': 'calendar#event', 'etag': '"3386889592648000"', 'id': '7je0lph6k52uknelid0an4h8lf_20230831T143000Z', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=N2plMGxwaDZrNTJ1a25lbGlkMGFuNGg4bGZfMjAyMzA4MzFUMTQzMDAwWiBjaGFybGVzbWluZ0BiZXJrZWxleS5lZHU', 'created': '2023-06-11T23:27:48.000Z', 'updated': '2023-08-31T01:19:56.324Z', 'summary': 'morning routine; weigh; affirmation. read; answer messages', 'description': '<p>teeth/face, weigh, water, dress, hair, pack bag</p>', 'creator': {'email': 'charlesming@berkeley.edu', 'self': True}, 'organizer': {'email': 'charlesming@berkeley.edu', 'self': True}, 'start': {'dateTime': '2023-08-31T07:30:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2023-08-31T08:15:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'recurringEventId': '7je0lph6k52uknelid0an4h8lf_R20230828T143000', 'originalStartTime': {'dateTime': '2023-08-31T07:30:00-07:00', 'timeZone': 

In [4]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# raw_documents = TextLoader('../../../state_of_the_union.txt').load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# documents = text_splitter.split_documents(raw_documents)
# db = Chroma.from_documents(documents, OpenAIEmbeddings())

"""
# plan: load each outfit as a separate document where the content contains all key data
"The retriever can filter text chunks based on metadata tags prior to performing semantic search. 
By defining metadata fields and their corresponding attributes, you can perform queries that consider both the text content and the metadata of the document.
The only time I see this being useful is to filter out outfits with a piece in the laundry."""
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
import json
from langchain.document_loaders import JSONLoader
from langchain.schema.document import Document
from closet.api import get_all_outfits

outfits = get_all_outfits()
documents = [Document(page_content=outfit.generate_description(), metadata={"id": outfit.id}) for outfit in outfits]
print(documents)
print(len(documents))

#TODO: Add metadata for ids of pieces in the outfit to filter out those in laundry.
# Define the metadata extraction function.
# def metadata_func(record: dict, metadata: dict) -> dict:

#     metadata["style"] = record.get("sender_name")
#     metadata["timestamp_ms"] = record.get("timestamp_ms")

#     return metadata

# filepath='./outfits.json'
# loader = JSONLoader(file_path=filepath, jq_schema='.outfits[]', content_key="description", metadata_func=metadata_func)
# data = loader.load()
# pprint(data)


[Document(page_content="Outfit: Polo Ralph Lauren shirt in blue, size M | Levi's 501 Original Fit Jeans pants in black, size 32 | Nike Killshot 2 Leather Shoes shoes in white, size 10 | Seiko Men's Stainless Steel Watch watch in silver, size N/A | Style: casual | Traits: casual, relaxed | Convenience Score: 8 | Like Score: 9", metadata={'id': 1}), Document(page_content="Outfit: Calvin Klein Cotton Classics shirt in white, size M | Under Armour Men's Raid 10-inch Workout Gym Shorts shorts in khaki, size 32 | Adidas Ultraboost 21 shoes in black, size 10 | Adidas Originals Cap hat in black, size L | Ray-Ban Classic Aviator sunglasses in black, size N/A | Style: beach | Traits: casual, relaxed, cozy | Convenience Score: 9 | Like Score: 7", metadata={'id': 2}), Document(page_content="Outfit: Polo Ralph Lauren shirt in blue, size M | Levi's 501 Original Fit Jeans pants in black, size 32 | Nike Killshot 2 Leather Shoes shoes in white, size 10 | Tommy Hilfiger Men's Classic Hooded Puffer Jacke

In [5]:
# No need to use splitter because chunk size is already good (granularity is per outfit)
# Create embeddings
db = Chroma.from_documents(documents, OpenAIEmbeddings())
retriever = db.as_retriever()

# sanity check
query = "What are three outfits for hanging out with friends?"
docs = db.similarity_search(query)
print(docs[0].page_content)
print(docs)
print('\n'.join(doc.page_content for doc in docs))

# Eventually query using weather and event context. Output goes into later chain to generate the full report.

Outfit: Polo Ralph Lauren shirt in blue, size M | Levi's 501 Original Fit Jeans pants in black, size 32 | Nike Killshot 2 Leather Shoes shoes in white, size 10 | Tommy Hilfiger Men's Classic Hooded Puffer Jacket jacket in brown, size M | Ray-Ban Classic Aviator sunglasses in black, size N/A | Style: casual | Traits: relaxed, casual | Convenience Score: 9 | Like Score: 8
[Document(page_content="Outfit: Polo Ralph Lauren shirt in blue, size M | Levi's 501 Original Fit Jeans pants in black, size 32 | Nike Killshot 2 Leather Shoes shoes in white, size 10 | Tommy Hilfiger Men's Classic Hooded Puffer Jacket jacket in brown, size M | Ray-Ban Classic Aviator sunglasses in black, size N/A | Style: casual | Traits: relaxed, casual | Convenience Score: 9 | Like Score: 8", metadata={'id': 11}), Document(page_content="Outfit: Polo Ralph Lauren shirt in blue, size M | Under Armour Men's Raid 10-inch Workout Gym Shorts shorts in khaki, size 32 | Nike Killshot 2 Leather Shoes shoes in white, size 10 |

In [14]:
from langchain.prompts.chat import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

human_input = f"""The weather forecast for today is {forecast_data}.
The next ten events in the user's calendar are {events}.
Provide two sentences of insight into the weather. Then, brief me on the day's events. Assume I have a general idea, and want to know if it looks busy, stressful, or any other standouts. 
Given that context, recommend three outfits to me, with a two sentence justification of each one. If possible, suggest times when an outfit change may be needed due to my schedule.
You have creative freedom to give other suggestions that will reduce my decision fatigue for the day."""

human_message_prompt = HumanMessagePromptTemplate(
        prompt = PromptTemplate(
            input_variables=["weather", "events"],
            template="""The weather forecast for today is {weather}.
            The next ten events in the user's calendar are {events}.
            Provide two sentences of insight into the weather. Then brief the user on the day's events. Assume he has a general idea, and wants to know if it looks busy, stressful, or any other standouts. 
            Given that context, recommend three outfits to the user, with a two sentence explanation of each one. If possible, suggest times when an outfit change may be needed due to the user's schedule.
            You have creative freedom to give other suggestions that will reduce my decision fatigue for the day.""",
        )
    )

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=f"You are a flirty assistant for {user}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke)."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])
    
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(llm=chat, prompt=prompt, verbose=True, memory=memory)
out = chain.predict(human_input=human_input)
print(out)






> Entering new LLMChain chain...
Prompt after formatting:
System: You are a flirty assistant for {'name': 'Charles', 'age': 21, 'sex': 'male'}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke).
Human: The weather forecast for today is {'latitude': 37.877266, 'longitude': -122.275635, 'generationtime_ms': 0.21195411682128906, 'utc_offset_seconds': -25200, 'timezone': 'America/Los_Angeles', 'timezone_abbreviation': 'PDT', 'elevation': 56.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°F', 'precipitation_probability': '%', 'precipitation': 'mm'}, 'hourly': {'time': ['2023-08-31T00:00', '2023-08-31T01:00', '2023-08-31T02:00', '2023-08-31T03:00', '2023-08-31T04:00', '2023-08-31T05:00', '2023-08-31T06:00', '2023-08-31T07:00', '2023-08-31T08:00', '2023-08-31T09:00', '2023-08-31T10:00', '2023-08-31T11:00', '2023-08-31T12:00', '2023-08-31T13:00', '2023-08-31T14:00', '2023-08-31T15:00', '2023-08-31T16:00', '2023-08-31T17:00', '2023-0


> Finished chain.
Well, it looks like the weather today in your area is going to be pretty pleasant with a high of 70.6°F and a low of 57.3°F. So you can expect a comfortable day ahead, not too hot and not too cold. 

As for your day's events, it seems like you have a busy morning with your routine, weighing yourself, affirmations, reading, and answering messages. Then you have a block of time dedicated to building something, followed by a snack break. After that, you hit the gym for a push and mobility session. Later in the day, you have lunch and some family face time. And to end the day, you have a mindful meal. Overall, it seems like a well-rounded day with a mix of personal care, productivity, and family time.

Now, let's talk about outfits! For the morning routine and building block, I recommend a comfortable and casual outfit like a pair of jeans, a t-shirt, and sneakers. This will give you the flexibility and ease of movement you need for your activities. You can switch to a m

In [15]:
chain.predict(human_input="Outfit 2 is currently in the laundry. What is an alternative?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a flirty assistant for {'name': 'Charles', 'age': 21, 'sex': 'male'}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke).
Human: The weather forecast for today is {'latitude': 37.877266, 'longitude': -122.275635, 'generationtime_ms': 0.21195411682128906, 'utc_offset_seconds': -25200, 'timezone': 'America/Los_Angeles', 'timezone_abbreviation': 'PDT', 'elevation': 56.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°F', 'precipitation_probability': '%', 'precipitation': 'mm'}, 'hourly': {'time': ['2023-08-31T00:00', '2023-08-31T01:00', '2023-08-31T02:00', '2023-08-31T03:00', '2023-08-31T04:00', '2023-08-31T05:00', '2023-08-31T06:00', '2023-08-31T07:00', '2023-08-31T08:00', '2023-08-31T09:00', '2023-08-31T10:00', '2023-08-31T11:00', '2023-08-31T12:00', '2023-08-31T13:00', '2023-08-31T14:00', '2023-08-31T15:00', '2023-08-31T16:00', '2023-08-31T17:00', '2023-0

"No worries! If Outfit 2 is currently in the laundry, I've got another suggestion for you. How about going for a sporty and comfortable look? You can wear a pair of joggers, a graphic t-shirt, and sneakers. This outfit will give you a relaxed and casual vibe, perfect for your gym session and other activities. Plus, it's always good to have a backup sporty outfit in your wardrobe for days like this!\n\nRemember, confidence is key, no matter what outfit you choose. Own it and rock it!"